### The observation of the agents consists of  
    1: the environment time, 2:all drivers and orders currently in the simulation. At time t there is a collection of orders oit ∈ Ot, drivers djt ∈ Dt. The state is given to the neural network as s = (O , D , t). The orders are represented as 6 dimensional vectors consisting of the starting and ending coordinates, price, and time waiting. Time waiting is the difference between the current time and the creation time. A driver is represented by a 6 dimensional vector: the coordinates of the driver location, x and y components of its reposition direction, time to order completion, and time to reposition completion. If the driver is serving an order, its location is set to the ending location of the order it is servicing. If the driver is repositioning the driver location is updated at each timestep, the reposition direction shows which way the driver will move during the next timestep. 
    
### Neural Network Architecture
    We propose a neural network architecture for RL in environ- ments with variable sized observation and action spaces.
    
### The assignment problem is one specific area within logistics that deals with the following question: 
    1: which Dasher should fulfill which delivery? We aim to make Dasher-delivery assignments that yield quick delivery speeds and healthy Dasher efficiency, where efficiency is number of deliveries performed by Dasher per unit time. In making these decisions, we need to consider many factors, including but not limited to:
    2: Consumer quoted times
    3: Estimated order ready times
    4: Travel estimations
    5: Routing (for multi-delivery assignments)
    6: Dasher utilization (percentage of time a Dasher is actively working on a delivery over an entire shift duration)
    7: Reward: Combination of delivery speeds and Dasher efficiency. The reward also includes a penalty for undelivered deliveries
    For our problem, the features generated from the states are intended to capture the information about deliveries and Dashers that are useful for predicting the Q-value, which are future delivery speeds and Dasher efficiencies. Some examples of these features are the pick-up to drop-off distances, the ratio of deliveries to Dashers, and the estimated order ready times

In [12]:
import ray
from ray import tune

from ray.rllib.agents import ddpg, ppo, dqn
from ray.tune.logger import pretty_print

from Envs import VRP as e
import importlib
importlib.reload(e)

<module 'Envs.VRP' from '/Users/mingjunwang/Documents/Backup/RL/Envs/VRP.py'>

In [ ]:
if __name__ == '__main__':
    
    stop = {
            "training_iteration": 40,
            "episode_reward_mean": 10000000,
        }

    config = {
            "env": e.SimpleSupplyChain, 
            # Use GPUs iff `RLLIB_NUM_GPUS` env var set to > 0.
            "num_gpus": 0,
            "num_workers": 4 , # parallelism,
            'framework': 'tf'
        }
    
    tune.run('DDPG',num_samples=2,
        stop=stop,
        config=config
    )

In [14]:
ray.shutdown()
ray.init()

def train_ppo():
    config = ppo.DEFAULT_CONFIG.copy()
    config["log_level"] = "INFO"
    #config["actor_hiddens"] = [512, 512] 
    #config["critic_hiddens"] = [512, 512]
    config["gamma"] = 0.99
    #config["timesteps_per_iteration"] = 1000
    #config["target_network_update_freq"] = 5
    #config["buffer_size"] = 10000
    config['framework'] = 'tf'
    config['vf_clip_param'] = 50000
    
    trainer = ppo.PPOTrainer(config=config, env=e.VRPGymEnvironment)
    for i in range(200):
        pass
        result = trainer.train()
        if i % 10 == 0:
            print("iteration: ", i)
        #print(pretty_print(result))
        #checkpoint = trainer.save()
        #print("Checkpoint saved at", checkpoint)

train_ppo()


2021-05-10 20:51:38,767	WARNING worker.py:622 -- File descriptor limit 256 is too low for production servers and may result in connection errors. At least 8192 is recommended. --- Fix with 'ulimit -n 8192'
2021-05-10 20:51:39,279	INFO services.py:1171 -- View the Ray dashboard at http://127.0.0.1:8266
2021-05-10 20:51:41,213	INFO trainer.py:591 -- Tip: set framework=tfe or the --eager flag to enable TensorFlow eager execution
(pid=71999) WARNING:tensorflow:From /Users/mingjunwang/opt/miniconda3/lib/python3.8/site-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=71999) Instructions for updating:
(pid=71999) non-resource variables are not supported in the long term
(pid=72001) WARNING:tensorflow:From /Users/mingjunwang/opt/miniconda3/lib/python3.8/site-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.

RayTaskError(AssertionError): [36mray::RolloutWorker.par_iter_next()[39m (pid=71999, ip=192.168.0.23)
  File "python/ray/_raylet.pyx", line 463, in ray._raylet.execute_task
  File "python/ray/_raylet.pyx", line 415, in ray._raylet.execute_task.function_executor
  File "/Users/mingjunwang/opt/miniconda3/lib/python3.8/site-packages/ray/util/iter.py", line 1152, in par_iter_next
    return next(self.local_it)
  File "/Users/mingjunwang/opt/miniconda3/lib/python3.8/site-packages/ray/rllib/evaluation/rollout_worker.py", line 323, in gen_rollouts
    yield self.sample()
  File "/Users/mingjunwang/opt/miniconda3/lib/python3.8/site-packages/ray/rllib/evaluation/rollout_worker.py", line 638, in sample
    batches = [self.input_reader.next()]
  File "/Users/mingjunwang/opt/miniconda3/lib/python3.8/site-packages/ray/rllib/evaluation/sampler.py", line 94, in next
    batches = [self.get_data()]
  File "/Users/mingjunwang/opt/miniconda3/lib/python3.8/site-packages/ray/rllib/evaluation/sampler.py"